In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from pylab import rcParams
rcParams['figure.figsize'] = 8, 6

df = pd.read_csv("../data/CheckPoint/training.csv", 
                 index_col="EventId")
df.head()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
EventId,,,,,,,,,,,,,,,,,,,,,
100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [2]:
df.drop(['Weight'], axis=1,inplace=True)#удаляем  столбец Weight
df.head()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Label
EventId,,,,,,,,,,,,,,,,,,,,,
100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,...,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,s
100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,...,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,b
100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,...,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,b
100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,...,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,b
100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,...,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,b


# Наивный байесовский классификатор, Решающее дерево и Метод k-ближайших соседей

In [3]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelBinarizer,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline


train = df.drop(['Label'], axis=1).values #получаем значения всех столбцов кроме Label
target = df["Label"].values # получаем значения столбца Label

#переводим значения s в 1, а b в 0
lb = LabelBinarizer()
target = np.array([number[0] for number in lb.fit_transform(target)])
#классификаторы
classifiers = [
    ("NB", GaussianNB(), {}),
    ("Tree", DecisionTreeClassifier(), {"max_depth": np.arange(2, 10)}),
    ("Knn", make_pipeline(
        StandardScaler(), 
        KNeighborsClassifier()),  {"kneighborsclassifier__n_neighbors": np.arange(7, 30, 2)}),
]

results = []
#вычисляем значения для каждого классификатора
for name, classifier, params in classifiers:    
    print(name)
    #оцениваем точность
    grid_acc = GridSearchCV(
        classifier, params, scoring='accuracy',
        cv=3, refit=False,        
    )
    grid_acc.fit(train, target)
    print("\tAcc =", grid_acc.best_score_, grid_acc.best_params_)
    #вычисляем F меру
    grid_f1 = GridSearchCV(
        classifier, params, scoring='f1',
        cv=3, refit=False,        
    )
    grid_f1.fit(train, target)
    print("\tF1 =", grid_f1.best_score_, grid_f1.best_params_)
    #считаем площадь под кривой ошибок
    grid_roc = GridSearchCV(
        classifier, params, scoring='roc_auc',
        cv=3, refit=False,        
    )
    grid_roc.fit(train, target)
    print("\tROC AUC =", grid_roc.best_score_, grid_roc.best_params_)
    
    results.append((
        name, 
        grid_f1.best_score_, grid_acc.best_score_, grid_roc.best_score_,
    ))
dr = pd.DataFrame(results, columns=["Name", "F1", "Accuracy", "ROC AUC"])


dr

NB
	Acc = 0.676312 {}
	F1 = 0.5365974511276186 {}
	ROC AUC = 0.7360082622595222 {}
Tree
	Acc = 0.820672 {'max_depth': 9}
	F1 = 0.7258677597578956 {'max_depth': 8}
	ROC AUC = 0.8832488721153087 {'max_depth': 9}
Knn
	Acc = 0.799288 {'kneighborsclassifier__n_neighbors': 27}
	F1 = 0.6990537266385547 {'kneighborsclassifier__n_neighbors': 27}
	ROC AUC = 0.8665389999623052 {'kneighborsclassifier__n_neighbors': 29}


,Name,F1,Accuracy,ROC AUC
0,NB,0.536597,0.676312,0.736008
1,Tree,0.725868,0.820672,0.883249
2,Knn,0.699054,0.799288,0.866539
